In [1]:
import requests

In [ ]:
# Buscar
user_localization = [40.4165000, -3.7025600]

In [36]:
import requests

# Define the endpoint URL for train localization
endpoint_url = 'https://dadesobertes.fgc.cat/api/records/1.0/search/'

# Set up the parameters
params = {
    'dataset': 'posicionament-dels-trens',
    'rows': 100,  # Number of records to fetch
    'q': 'lin:S2'  # Replace LINE_ID with the specific line you want to filter by
}

# Make the request
response = requests.get(endpoint_url, params=params)

# Check if the response is successful
if response.status_code == 200:
    # Parse the JSON response
    train_data = response.json()       
else:
    print("Error:", response.status_code, response.text)


In [33]:
train_data['records'][0]['fields'].keys()

dict_keys(['ocupacio_m2_tram', 'lin', 'id', 'ocupacio_m1_tram', 'ocupacio_ri_tram', 'ocupacio_m2_percent', 'geo_point_2d', 'ocupacio_m1_percent', 'ocupacio_mi_percent', 'ocupacio_ri_percent', 'ocupacio_mi_tram', 'origen', 'dir', 'properes_parades', 'desti', 'en_hora', 'estacionat_a', 'ut', 'tipus_unitat'])

In [ ]:
train_data['records'][0]['fields']

In [41]:
trains = []
for data in train_data['records']:   
    geo = data['fields']['geo_point_2d']
    trains.append({'id': data['fields']['id'], 'lat': geo[0], 'lon': geo[1]})

In [42]:
import math

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in km
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat / 2) ** 2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

# User location
user_lat = 41.3851  # Example latitude
user_lon = 2.1734   # Example longitude

# Find the nearest train
nearest_train = None
min_distance = float('inf')

for train in trains:
    distance = haversine(user_lat, user_lon, train['lat'], train['lon'])
    if distance < min_distance:
        min_distance = distance
        nearest_train = train

print(f"The nearest train is {nearest_train['id']} at a distance of {min_distance:.2f} km")


The nearest train is 6c4bdab144746c0fbf55c60d40|682dc7e005 at a distance of 2.95 km


In [76]:
def get_trains(rows = 100, line = 'S2'):
    # Define the endpoint URL for train localization
    endpoint_url = 'https://dadesobertes.fgc.cat/api/records/1.0/search/'

    # Set up the parameters
    params = {
        'dataset': 'posicionament-dels-trens',
        'rows': rows,  # Number of records to fetch
        'q': f'lin:{line}'  # Replace LINE_ID with the specific line you want to filter by
    }

    # Make the request
    response = requests.get(endpoint_url, params=params)

    # Check if the response is successful
    if response.status_code == 200:
        # Parse the JSON response
        train_data = response.json()       
    else:
        print("Error:", response.status_code, response.text)
    return train_data

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in km
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat / 2) ** 2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def get_my_train(user_lat, user_lon, trains_information):
    trains_info = []
    for data in trains_information['records']:   
        geo = data['fields']['geo_point_2d']
        trains_info.append({'id': data['fields']['id'], 'lat': geo[0], 'lon': geo[1]})
    
    nearest_train = None
    min_distance = float('inf')

    for train in trains_info:
        distance = haversine(user_lat, user_lon, train['lat'], train['lon'])
        if distance < min_distance:
            min_distance = distance
            nearest_train = train
            
    return nearest_train

def get_train(user_lat, user_lon):
    trains_information = get_trains()
    nearest_train = get_my_train(user_lat = user_lat, 
                            user_lon = user_lon, 
                            trains_information = trains_information)
    # Get my train
    for train in trains_information['records']:
        if train['fields']['id'] == nearest_train['id']:
            my_train = train
            break
    return my_train
    
my_train = get_train(user_lat = 41.3851, user_lon = 2.1734)

In [85]:
next_stations = [eval(parada)['parada'] for parada in my_train['fields']['properes_parades'].split(';')]

['PR',
 'GR',
 'SG',
 'MN',
 'BN',
 'TT',
 'SR',
 'PF',
 'VL',
 'LP',
 'LF',
 'VD',
 'SC',
 'VO',
 'SJ',
 'BT',
 'UN',
 'SQ',
 'CF',
 'PJ',
 'CT',
 'NO',
 'PN']

In [93]:
import requests

# Define the endpoint URL for the FGC stations dataset
endpoint_url = 'https://dadesobertes.fgc.cat/api/records/1.0/search/'

# Function to get the full name based on station code
def get_station_name_by_code(station_code):
    # Set up the parameters
    params = {
        'dataset': 'codigo-estaciones',
        'q': station_code,
        'rows': 1  # Assuming station codes are unique, fetch one record
    }

    # Make the request
    response = requests.get(endpoint_url, params=params)

    # Check if the response is successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        if data['records']:
            fields = data['records'][0]['fields']
            return fields['nom_estacio']
        else:
            return "Station not found"
    else:
        return f"Error: {response.status_code}, {response.text}"

# Example usage
station_code = 'PR'
station_name = get_station_name_by_code(station_code)
print(f"The full name for station code {station_code} is: {station_name}")


The full name for station code PR is: Provença
